### 1 - Carregar os pactotes e configurações do ambiente python


In [ ]:
import os.path as osp
import torch
from gnnepcsaft.data.graphdataset import ThermoMLDataset, Esper
from gnnepcsaft.train.models import GNNePCSAFTL
from gnnepcsaft.data.graph import from_InChI
from gnnepcsaft.configs.default import get_config
from gnnepcsaft.demo.utils import plotdata
import networkx as nx
from torch_geometric.utils import to_networkx
from torch_geometric.nn import summary

config = get_config()
device = "cpu"
model_dtype = torch.float64

print(torch.cuda.is_available())

### 2 - Carregar os dados de parametrização e dados experimentais de densidade e pressão de vapor


- Dados de parametrização


In [ ]:
es = Esper("gnnepcsaft/data/esper2023")
es_para = {}
for graph in es:
    inchi, para = graph.InChI, graph.para
    es_para[inchi] = para

- Dados de pressão de vapor e densidade


In [ ]:
path = osp.join("gnnepcsaft/data", "thermoml")
tml = ThermoMLDataset(path)

### 3 - Criar e inicializar a Graph Neural Network


In [ ]:
model2 = GNNePCSAFTL.load_from_checkpoint(
    "gnnepcsaft/train/checkpoints/pna_msigmae_1.0-epoch=19999-mape_den.ckpt", "cpu"
).eval()


model1 = GNNePCSAFTL.load_from_checkpoint(
    "gnnepcsaft/train/checkpoints/pna_msigmae_1.0-epoch=21249-mape_den.ckpt", "cpu"
).eval()

- Resumo dos parâmetros do modelo


In [ ]:
dummy = tml[0].x, tml[0].edge_index, tml[0].edge_attr, tml[0].batch  # type: ignore
print(summary(model2, *dummy, max_depth=3))

### 4 - Fazer a predição dos parâmetros a partir dos grafos


- InChI da molécula


In [ ]:
inchi = "InChI=1S/C5H6O2/c6-4-5-2-1-3-7-5/h1-3,6H,4H2"
# inchi = "InChI=1S/H2O/h1H2"

- Grafo da molécula


In [ ]:
grafo = from_InChI(inchi).to(device)
print("###### Nós (átomos) do grafo e seus atributos ###### \n")
print(grafo.x)
print("\n###### Arestas (ligações químicas) da molécula e seus atributos ######\n")
print(grafo.edge_attr)
print("\n###### Figura do grafo ######\n")
nxg = to_networkx(grafo)
nx.draw(nxg)

- Parâmetros previstos pelos Modelo 1 e 2


In [ ]:
with torch.no_grad():
    input = grafo.x, grafo.edge_index, grafo.edge_attr, grafo.batch

    parametros_modelo_1 = model1(*input)

    parametros_modelo_2 = model2(*input)


print("######### Parâmetros previstos #########\n")


print("Pelo Modelo 1:\n")


print("[número de segmentos, diâmetros de segmento (Å), energia de dispersão (K)]")


print(parametros_modelo_1[0].numpy().round(5))


print("\nPelo Modelo 2:\n")


print("[número de segmentos, diâmetros de segmento (Å), energia de dispersão (K)]")


print(parametros_modelo_2[0].numpy().round(5))

- Figuras demonstrando o desempenho dos parâmetros


In [ ]:
molecule_name = "furfuranol"
print(f"######### {molecule_name} #########")
plotdata(inchi, molecule_name, models=[model1.model, model2.model], model_msigmae=None)